In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\andy\anaconda3\envs\pythondata\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd
from numpy.random import seed
seed(99)
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
df.columns

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')

# Select your features (columns)

In [6]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [7]:
target = df['koi_disposition']
target.unique()

array(['CONFIRMED', 'FALSE POSITIVE', 'CANDIDATE'], dtype=object)

In [8]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(target)
encoded_y = label_encoder.transform(target)
encoded_y

array([1, 2, 2, ..., 0, 2, 2])

In [9]:
from keras.utils import to_categorical

# Step 2: One-hot encoding
one_hot_y = to_categorical(encoded_y)
one_hot_y

Using TensorFlow backend.


array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(selected_features, one_hot_y, random_state=99)

In [11]:
X_train.head()

,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth
5136,94.381630,171.336200,0.5930,11.87000,91.2
6214,24.238242,136.616569,0.4260,4.16665,461960.0
5863,2.705816,132.537260,0.6199,6.56000,46.9
2100,8.589696,174.962500,1.1970,8.47100,245.1
3977,2.401752,133.711919,5.2500,2.08350,25044.0


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [12]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
X_minmax = MinMaxScaler().fit(X_train)
y_minmax = MinMaxScaler().fit(y_train)

X_train_minmax = X_minmax.transform(X_train)
X_test_minmax = X_minmax.transform(X_test)
y_train_minmax = y_minmax.transform(y_train)
y_test_minmax = y_minmax.transform(y_test)

# Train the Model



In [13]:
model = Sequential()
model.add(Dense(units=12, activation='relu', input_dim=5))
model.add(Dense(units=12, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [14]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(
    X_train_minmax,
    y_train_minmax,
    epochs=500,
    shuffle=True,
    verbose=2)

Train on 5243 samples
Epoch 1/500
5243/5243 - 1s - loss: 1.0539 - accuracy: 0.4875
Epoch 2/500
5243/5243 - 0s - loss: 1.0144 - accuracy: 0.5039
Epoch 3/500
5243/5243 - 0s - loss: 0.9902 - accuracy: 0.5039
Epoch 4/500
5243/5243 - 0s - loss: 0.9643 - accuracy: 0.5039
Epoch 5/500
5243/5243 - 0s - loss: 0.9393 - accuracy: 0.5035
Epoch 6/500
5243/5243 - 0s - loss: 0.9209 - accuracy: 0.5110
Epoch 7/500
5243/5243 - 0s - loss: 0.9075 - accuracy: 0.5251
Epoch 8/500
5243/5243 - 0s - loss: 0.8960 - accuracy: 0.5463
Epoch 9/500
5243/5243 - 0s - loss: 0.8877 - accuracy: 0.5567
Epoch 10/500
5243/5243 - 0s - loss: 0.8797 - accuracy: 0.5699
Epoch 11/500
5243/5243 - 0s - loss: 0.8747 - accuracy: 0.5737
Epoch 12/500
5243/5243 - 0s - loss: 0.8688 - accuracy: 0.5810
Epoch 13/500
5243/5243 - 0s - loss: 0.8635 - accuracy: 0.5878
Epoch 14/500
5243/5243 - 0s - loss: 0.8583 - accuracy: 0.5922
Epoch 15/500
5243/5243 - 0s - loss: 0.8546 - accuracy: 0.5916
Epoch 16/500
5243/5243 - 0s - loss: 0.8503 - accuracy: 0.

Epoch 133/500
5243/5243 - 0s - loss: 0.7512 - accuracy: 0.6502
Epoch 134/500
5243/5243 - 0s - loss: 0.7505 - accuracy: 0.6534
Epoch 135/500
5243/5243 - 0s - loss: 0.7494 - accuracy: 0.6540
Epoch 136/500
5243/5243 - 0s - loss: 0.7503 - accuracy: 0.6538
Epoch 137/500
5243/5243 - 0s - loss: 0.7490 - accuracy: 0.6561
Epoch 138/500
5243/5243 - 0s - loss: 0.7493 - accuracy: 0.6573
Epoch 139/500
5243/5243 - 0s - loss: 0.7481 - accuracy: 0.6567
Epoch 140/500
5243/5243 - 0s - loss: 0.7460 - accuracy: 0.6567
Epoch 141/500
5243/5243 - 0s - loss: 0.7481 - accuracy: 0.6542
Epoch 142/500
5243/5243 - 0s - loss: 0.7484 - accuracy: 0.6544
Epoch 143/500
5243/5243 - 0s - loss: 0.7471 - accuracy: 0.6574
Epoch 144/500
5243/5243 - 0s - loss: 0.7481 - accuracy: 0.6555
Epoch 145/500
5243/5243 - 0s - loss: 0.7458 - accuracy: 0.6555
Epoch 146/500
5243/5243 - 0s - loss: 0.7456 - accuracy: 0.6573
Epoch 147/500
5243/5243 - 0s - loss: 0.7448 - accuracy: 0.6582
Epoch 148/500
5243/5243 - 0s - loss: 0.7449 - accuracy:

Epoch 264/500
5243/5243 - 0s - loss: 0.7147 - accuracy: 0.6677
Epoch 265/500
5243/5243 - 0s - loss: 0.7140 - accuracy: 0.6693
Epoch 266/500
5243/5243 - 0s - loss: 0.7130 - accuracy: 0.6742
Epoch 267/500
5243/5243 - 0s - loss: 0.7123 - accuracy: 0.6716
Epoch 268/500
5243/5243 - 0s - loss: 0.7129 - accuracy: 0.6718
Epoch 269/500
5243/5243 - 0s - loss: 0.7110 - accuracy: 0.6737
Epoch 270/500
5243/5243 - 0s - loss: 0.7115 - accuracy: 0.6718
Epoch 271/500
5243/5243 - 0s - loss: 0.7122 - accuracy: 0.6693
Epoch 272/500
5243/5243 - 0s - loss: 0.7123 - accuracy: 0.6706
Epoch 273/500
5243/5243 - 0s - loss: 0.7107 - accuracy: 0.6700
Epoch 274/500
5243/5243 - 0s - loss: 0.7117 - accuracy: 0.6702
Epoch 275/500
5243/5243 - 0s - loss: 0.7112 - accuracy: 0.6737
Epoch 276/500
5243/5243 - 0s - loss: 0.7107 - accuracy: 0.6693
Epoch 277/500
5243/5243 - 0s - loss: 0.7098 - accuracy: 0.6700
Epoch 278/500
5243/5243 - 0s - loss: 0.7123 - accuracy: 0.6714
Epoch 279/500
5243/5243 - 0s - loss: 0.7104 - accuracy:

5243/5243 - 0s - loss: 0.6950 - accuracy: 0.6813
Epoch 395/500
5243/5243 - 0s - loss: 0.6927 - accuracy: 0.6817
Epoch 396/500
5243/5243 - 0s - loss: 0.6939 - accuracy: 0.6817
Epoch 397/500
5243/5243 - 0s - loss: 0.6971 - accuracy: 0.6792
Epoch 398/500
5243/5243 - 0s - loss: 0.6939 - accuracy: 0.6767
Epoch 399/500
5243/5243 - 0s - loss: 0.6922 - accuracy: 0.6855
Epoch 400/500
5243/5243 - 0s - loss: 0.6933 - accuracy: 0.6821
Epoch 401/500
5243/5243 - 0s - loss: 0.6913 - accuracy: 0.6782
Epoch 402/500
5243/5243 - 0s - loss: 0.6927 - accuracy: 0.6792
Epoch 403/500
5243/5243 - 0s - loss: 0.6926 - accuracy: 0.6780
Epoch 404/500
5243/5243 - 0s - loss: 0.6917 - accuracy: 0.6809
Epoch 405/500
5243/5243 - 0s - loss: 0.6956 - accuracy: 0.6832
Epoch 406/500
5243/5243 - 0s - loss: 0.6966 - accuracy: 0.6769
Epoch 407/500
5243/5243 - 0s - loss: 0.6929 - accuracy: 0.6817
Epoch 408/500
5243/5243 - 0s - loss: 0.6918 - accuracy: 0.6832
Epoch 409/500
5243/5243 - 0s - loss: 0.6904 - accuracy: 0.6815
Epoch 

In [16]:
model_loss, model_accuracy = model.evaluate(X_train_minmax, y_train_minmax, verbose=2)
print(f"Training Data Loss: {model_loss}, Training Data Accuracy: {model_accuracy}")

model_loss, model_accuracy = model.evaluate(X_test_minmax, y_test_minmax, verbose=2)
print(f"Test Data Loss: {model_loss}, Test Data Accuracy: {model_accuracy}")

5243/5243 - 0s - loss: 0.6800 - accuracy: 0.6925
Training Data Loss: 0.679995248489609, Training Data Accuracy: 0.6925424337387085
1748/1748 - 0s - loss: 0.6848 - accuracy: 0.7014
Test Data Loss: 0.6847924400521634, Test Data Accuracy: 0.7013729810714722


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [17]:
model

# Save the Model

In [18]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
# import joblib
# filename = 'andy_spatz.sav'
# neuralnet = model
# joblib.dump(neuralnet, filename)

model.save('neuralnetmodel.h5')